In [1]:
import numpy as np
import pandas as pd

from numpy import array

In [2]:
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq

In [3]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], np.transpose(inv_mat)))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [4]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneHot(sequence_nt))
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],50,4,1))
X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = find_c0new(data_tiling).astype(float)


In [5]:
def find_pooling_by_5_scores(one_hot_seq, indices_of_interest=[0,3], offset=0):
    scores = []
    if offset != 0:
        cur_region = one_hot_seq[:offset]
        scores.append(2*np.sum(cur_region[:,indices_of_interest])-len(cur_region))
    for i in range(len(one_hot_seq)//5):
        cur_region = one_hot_seq[(i*5)+offset:((i+1)*5)+offset]
        # print(cur_region.shape)
        scores.append(2*np.sum(cur_region[:,indices_of_interest])-len(cur_region))
    return scores

In [30]:
indices_of_interest = [0,3]

X5_pooling_by_5_scores_offset0 = []
for one_hot_seq in X5:
    X5_pooling_by_5_scores_offset0.append(find_pooling_by_5_scores(one_hot_seq, indices_of_interest=indices_of_interest))
X5_pooling_by_5_scores_offset0 = array(X5_pooling_by_5_scores_offset0)

X5_pooling_by_5_scores_offset1 = []
for one_hot_seq in X5:
    X5_pooling_by_5_scores_offset1.append(find_pooling_by_5_scores(one_hot_seq, indices_of_interest=indices_of_interest, offset=1))
X5_pooling_by_5_scores_offset1 = array(X5_pooling_by_5_scores_offset1)

X5_pooling_by_5_scores_offset2 = []
for one_hot_seq in X5:
    X5_pooling_by_5_scores_offset2.append(find_pooling_by_5_scores(one_hot_seq, indices_of_interest=indices_of_interest, offset=2))
X5_pooling_by_5_scores_offset2 = array(X5_pooling_by_5_scores_offset2)

X5_pooling_by_5_scores_offset3 = []
for one_hot_seq in X5:
    X5_pooling_by_5_scores_offset3.append(find_pooling_by_5_scores(one_hot_seq, indices_of_interest=indices_of_interest, offset=3))
X5_pooling_by_5_scores_offset3 = array(X5_pooling_by_5_scores_offset3)

X5_pooling_by_5_scores_offset4 = []
for one_hot_seq in X5:
    X5_pooling_by_5_scores_offset4.append(find_pooling_by_5_scores(one_hot_seq, indices_of_interest=indices_of_interest, offset=4))
X5_pooling_by_5_scores_offset4 = array(X5_pooling_by_5_scores_offset4)

## Alternating Scores (Periodicty):

In [391]:
def find_alternating_score(scores):
    odd_scores = scores[range(0,len(scores),2)]
    even_scores = scores[range(1,len(scores),2)]
    return(abs(sum(odd_scores) - sum(even_scores)))

In [392]:
X5_global_alternating_scores_offset0 = []
for scores in X5_pooling_by_5_scores_offset0:
    X5_global_alternating_scores_offset0.append(find_alternating_score(scores))
X5_global_alternating_scores_offset0 = array(X5_global_alternating_scores_offset0)

X5_global_alternating_scores_offset1 = []
for scores in X5_pooling_by_5_scores_offset1:
    X5_global_alternating_scores_offset1.append(find_alternating_score(scores))
X5_global_alternating_scores_offset1 = array(X5_global_alternating_scores_offset1)

X5_global_alternating_scores_offset2 = []
for scores in X5_pooling_by_5_scores_offset2:
    X5_global_alternating_scores_offset2.append(find_alternating_score(scores))
X5_global_alternating_scores_offset2 = array(X5_global_alternating_scores_offset2)

X5_global_alternating_scores_offset3 = []
for scores in X5_pooling_by_5_scores_offset3:
    X5_global_alternating_scores_offset3.append(find_alternating_score(scores))
X5_global_alternating_scores_offset3 = array(X5_global_alternating_scores_offset3)

X5_global_alternating_scores_offset4 = []
for scores in X5_pooling_by_5_scores_offset4:
    X5_global_alternating_scores_offset4.append(find_alternating_score(scores))
X5_global_alternating_scores_offset4 = array(X5_global_alternating_scores_offset4)

In [393]:
print(np.corrcoef(X5_global_alternating_scores_offset0, Y5)[0,1])
print(np.corrcoef(X5_global_alternating_scores_offset1, Y5)[0,1])
print(np.corrcoef(X5_global_alternating_scores_offset2, Y5)[0,1])
print(np.corrcoef(X5_global_alternating_scores_offset3, Y5)[0,1])
print(np.corrcoef(X5_global_alternating_scores_offset4, Y5)[0,1])

0.27011085101322885
0.27032852306138755
0.212597949185094
0.17599584935415466
0.20748425921092523


In [394]:
np.corrcoef(np.max(np.array([
    X5_global_alternating_scores_offset0, 
    X5_global_alternating_scores_offset1,
    X5_global_alternating_scores_offset2,
    X5_global_alternating_scores_offset3,
    X5_global_alternating_scores_offset4,
    ]), axis=0), Y5)[0,1]

0.36169119181132736

In [395]:
np.corrcoef(np.mean(np.array([
    X5_global_alternating_scores_offset0, 
    X5_global_alternating_scores_offset1,
    X5_global_alternating_scores_offset2,
    X5_global_alternating_scores_offset3,
    X5_global_alternating_scores_offset4,
    ]), axis=0), Y5)[0,1]

0.3927914986868216

In [396]:
np.corrcoef(np.min(np.array([
    X5_global_alternating_scores_offset0, 
    X5_global_alternating_scores_offset1,
    X5_global_alternating_scores_offset2,
    X5_global_alternating_scores_offset3,
    X5_global_alternating_scores_offset4,
    ]), axis=0), Y5)[0,1]

0.1456671929110105

In [397]:
def analyze_alternating(scores):
    total_times_alternated = 0
    cur_times_alternated = 0
    max_consecutive_times_alternated = 0
    cur_alternating_scores = []
    sum_local_alternating_scores = 0
    prev = 0
    for score in scores:
        # Check whether currently alternating:
        if (score > 0 and prev < 0) or (score < 0 and prev > 0):
            total_times_alternated += 1
            cur_times_alternated += 1
            cur_alternating_scores.append(score)
            if cur_times_alternated > max_consecutive_times_alternated:
                max_consecutive_times_alternated = cur_times_alternated
        else:
            if len(cur_alternating_scores) > 0:
                sum_local_alternating_scores+=find_alternating_score(array(cur_alternating_scores))*(len(cur_alternating_scores)-1)
                cur_alternating_scores = [score]
            cur_times_alternated = 0
        prev = score
    if len(cur_alternating_scores) > 1:
        sum_local_alternating_scores+=find_alternating_score(array(cur_alternating_scores))*len(cur_alternating_scores)
    return total_times_alternated, max_consecutive_times_alternated, sum_local_alternating_scores

In [398]:
idx = 0
analyze_alternating(X5_pooling_by_5_scores_offset0[idx]), X5_pooling_by_5_scores_offset0[idx], data_tiling.Sequence[idx]

((5, 2, 18.0),
 array([ 1.,  1., -1.,  3.,  1.,  1., -1.,  1.,  3., -1.]),
 'GTGATCTTCTCTTGCTCATTGCATTTGGAACTGAGACAGAAGAATGACGA')

In [399]:
X5_count_alternating_offset0 = []
X5_max_consecutive_alternating_offset0 = []
X5_sum_local_alternating_scores_offset0 = []
for scores in X5_pooling_by_5_scores_offset0:
    total_times, max_consecutive, sum_local_alternating_scores = analyze_alternating(scores)
    X5_count_alternating_offset0.append(total_times)
    X5_max_consecutive_alternating_offset0.append(max_consecutive)
    X5_sum_local_alternating_scores_offset0.append(sum_local_alternating_scores)
X5_count_alternating_offset0 = array(X5_count_alternating_offset0)
X5_max_consecutive_alternating_offset0 = array(X5_max_consecutive_alternating_offset0)
X5_sum_local_alternating_scores_offset0 = array(X5_sum_local_alternating_scores_offset0)

X5_count_alternating_offset1 = []
X5_max_consecutive_alternating_offset1 = []
X5_sum_local_alternating_scores_offset1 = []
for scores in X5_pooling_by_5_scores_offset1:
    total_times, max_consecutive, sum_local_alternating_scores = analyze_alternating(scores)
    X5_count_alternating_offset1.append(total_times)
    X5_max_consecutive_alternating_offset1.append(max_consecutive)
    X5_sum_local_alternating_scores_offset1.append(sum_local_alternating_scores)
X5_count_alternating_offset1 = array(X5_count_alternating_offset1)
X5_max_consecutive_alternating_offset1 = array(X5_max_consecutive_alternating_offset1)
X5_sum_local_alternating_scores_offset1 = array(X5_sum_local_alternating_scores_offset1)

X5_count_alternating_offset2 = []
X5_max_consecutive_alternating_offset2 = []
X5_sum_local_alternating_scores_offset2 = []
for scores in X5_pooling_by_5_scores_offset2:
    total_times, max_consecutive, sum_local_alternating_scores = analyze_alternating(scores)
    X5_count_alternating_offset2.append(total_times)
    X5_max_consecutive_alternating_offset2.append(max_consecutive)
    X5_sum_local_alternating_scores_offset2.append(sum_local_alternating_scores)
X5_count_alternating_offset2 = array(X5_count_alternating_offset2)
X5_max_consecutive_alternating_offset2 = array(X5_max_consecutive_alternating_offset2)
X5_sum_local_alternating_scores_offset2 = array(X5_sum_local_alternating_scores_offset2)

X5_count_alternating_offset3 = []
X5_max_consecutive_alternating_offset3 = []
X5_sum_local_alternating_scores_offset3 = []
for scores in X5_pooling_by_5_scores_offset3:
    total_times, max_consecutive, sum_local_alternating_scores = analyze_alternating(scores)
    X5_count_alternating_offset3.append(total_times)
    X5_max_consecutive_alternating_offset3.append(max_consecutive)
    X5_sum_local_alternating_scores_offset3.append(sum_local_alternating_scores)
X5_count_alternating_offset3 = array(X5_count_alternating_offset3)
X5_max_consecutive_alternating_offset3 = array(X5_max_consecutive_alternating_offset3)
X5_sum_local_alternating_scores_offset3 = array(X5_sum_local_alternating_scores_offset3)

X5_count_alternating_offset4 = []
X5_max_consecutive_alternating_offset4 = []
X5_sum_local_alternating_scores_offset4 = []
for scores in X5_pooling_by_5_scores_offset4:
    total_times, max_consecutive, sum_local_alternating_scores = analyze_alternating(scores)
    X5_count_alternating_offset4.append(total_times)
    X5_max_consecutive_alternating_offset4.append(max_consecutive)
    X5_sum_local_alternating_scores_offset4.append(sum_local_alternating_scores)
X5_count_alternating_offset4 = array(X5_count_alternating_offset4)
X5_max_consecutive_alternating_offset4 = array(X5_max_consecutive_alternating_offset4)
X5_sum_local_alternating_scores_offset4 = array(X5_sum_local_alternating_scores_offset4)

In [400]:
print(np.corrcoef(X5_count_alternating_offset0, Y5)[0,1])
print(np.corrcoef(X5_count_alternating_offset1, Y5)[0,1])
print(np.corrcoef(X5_count_alternating_offset2, Y5)[0,1])
print(np.corrcoef(X5_count_alternating_offset3, Y5)[0,1])
print(np.corrcoef(X5_count_alternating_offset4, Y5)[0,1])

0.08127688749723189
0.0779209058932672
0.06598008405505758
0.05828003736648095
0.06720783336055447


In [401]:
np.corrcoef(np.max(np.array([
    X5_count_alternating_offset0, 
    X5_count_alternating_offset1,
    X5_count_alternating_offset2,
    X5_count_alternating_offset3,
    X5_count_alternating_offset4,
    ]), axis=0), Y5)[0,1]

0.09868703716961548

In [402]:
np.corrcoef(np.mean(np.array([
    X5_count_alternating_offset0, 
    X5_count_alternating_offset1,
    X5_count_alternating_offset2,
    X5_count_alternating_offset3,
    X5_count_alternating_offset4,
    ]), axis=0), Y5)[0,1]

0.08801229745977872

In [403]:
np.corrcoef(np.min(np.array([
    X5_count_alternating_offset0, 
    X5_count_alternating_offset1,
    X5_count_alternating_offset2,
    X5_count_alternating_offset3,
    X5_count_alternating_offset4,
    ]), axis=0), Y5)[0,1]

0.053602435558827884

In [404]:
print(np.corrcoef(X5_max_consecutive_alternating_offset0, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_alternating_offset1, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_alternating_offset2, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_alternating_offset3, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_alternating_offset4, Y5)[0,1])

0.11934852272366231
0.1186121983611056
0.10251439935734291
0.08596306048424068
0.09584463202351289


In [405]:
np.corrcoef(np.max(np.array([
    X5_max_consecutive_alternating_offset0, 
    X5_max_consecutive_alternating_offset1,
    X5_max_consecutive_alternating_offset2,
    X5_max_consecutive_alternating_offset3,
    X5_max_consecutive_alternating_offset4,
    ]), axis=0), Y5)[0,1]

0.15937937487126655

In [406]:
np.corrcoef(np.mean(np.array([
    X5_max_consecutive_alternating_offset0, 
    X5_max_consecutive_alternating_offset1,
    X5_max_consecutive_alternating_offset2,
    X5_max_consecutive_alternating_offset3,
    X5_max_consecutive_alternating_offset4,
    ]), axis=0), Y5)[0,1]

0.15400228661225657

In [407]:
np.corrcoef(np.min(np.array([
    X5_max_consecutive_alternating_offset0, 
    X5_max_consecutive_alternating_offset1,
    X5_max_consecutive_alternating_offset2,
    X5_max_consecutive_alternating_offset3,
    X5_max_consecutive_alternating_offset4,
    ]), axis=0), Y5)[0,1]

0.06889356894900606

In [408]:
print(np.corrcoef(X5_sum_local_alternating_scores_offset0, Y5)[0,1])
print(np.corrcoef(X5_sum_local_alternating_scores_offset1, Y5)[0,1])
print(np.corrcoef(X5_sum_local_alternating_scores_offset2, Y5)[0,1])
print(np.corrcoef(X5_sum_local_alternating_scores_offset3, Y5)[0,1])
print(np.corrcoef(X5_sum_local_alternating_scores_offset4, Y5)[0,1])

0.13915606173442358
0.14808020790579365
0.11916131337842926
0.10042942342203523
0.10889183963574108


In [409]:
np.corrcoef(np.max(np.array([
    X5_sum_local_alternating_scores_offset0, 
    X5_sum_local_alternating_scores_offset1,
    X5_sum_local_alternating_scores_offset2,
    X5_sum_local_alternating_scores_offset3,
    X5_sum_local_alternating_scores_offset4,
    ]), axis=0), Y5)[0,1]

0.1973849229120663

In [410]:
np.corrcoef(np.mean(np.array([
    X5_sum_local_alternating_scores_offset0, 
    X5_sum_local_alternating_scores_offset1,
    X5_sum_local_alternating_scores_offset2,
    X5_sum_local_alternating_scores_offset3,
    X5_sum_local_alternating_scores_offset4,
    ]), axis=0), Y5)[0,1]

0.18577543264802668

In [411]:
np.corrcoef(np.min(np.array([
    X5_sum_local_alternating_scores_offset0, 
    X5_sum_local_alternating_scores_offset1,
    X5_sum_local_alternating_scores_offset2,
    X5_sum_local_alternating_scores_offset3,
    X5_sum_local_alternating_scores_offset4,
    ]), axis=0), Y5)[0,1]

0.07968650512233763

## Consecutive Scores (Rigidity)
### Note -  I should probably add something here to distinguish between consecutive *positive* scores and consecutive *negative* scores

In [412]:
def find_consecutive_score(scores):
    return((sum(scores)))

In [413]:
X5_global_consecutive_scores = []
for scores in X5_pooling_by_5_scores_offset0:
    X5_global_consecutive_scores.append(find_consecutive_score(scores))
X5_global_consecutive_scores = array(X5_global_consecutive_scores)

In [414]:
print(np.corrcoef(X5_global_consecutive_scores, Y5)[0,1])

-0.039794923384970336


In [415]:
def analyze_consecutive(scores):
    total_consecutive_count = 0
    total_consecutive_positive_count = 0
    total_consecutive_negative_count = 0
    cur_consecutive_count = 0
    cur_consecutive_positive_count = 0
    cur_consecutive_negative_count = 0
    max_consecutive_count = 0
    max_consecutive_positive_count = 0
    max_consecutive_negative_count = 0
    cur_consecutive_scores = []
    sum_local_consecutive_scores = 0
    sum_local_consecutive_positive_scores = 0
    sum_local_consecutive_negative_scores = 0
    prev = 0
    for score in scores:
        # Check whether currently consecutive:
        if (score > 0 and prev > 0):
            total_consecutive_count += 1
            cur_consecutive_count += 1
            cur_consecutive_scores.append(score)
            if cur_consecutive_count > max_consecutive_count:
                max_consecutive_count = cur_consecutive_count
            
            total_consecutive_positive_count += 1
            cur_consecutive_positive_count += 1
            if cur_consecutive_positive_count > max_consecutive_positive_count:
                max_consecutive_positive_count = cur_consecutive_positive_count
            
            cur_consecutive_negative_count = 0
        elif (score < 0 and prev < 0):
            total_consecutive_count += 1
            cur_consecutive_count += 1
            cur_consecutive_scores.append(score)
            if cur_consecutive_count > max_consecutive_count:
                max_consecutive_count = cur_consecutive_count
            
            total_consecutive_negative_count += 1
            cur_consecutive_negative_count += 1
            if cur_consecutive_negative_count > max_consecutive_negative_count:
                max_consecutive_negative_count = cur_consecutive_negative_count
            
            cur_consecutive_positive_count = 0
        else:
            if len(cur_consecutive_scores) > 1:
                local_consecutive_score = find_consecutive_score(array(cur_consecutive_scores))*(len(cur_consecutive_scores)-1)
                sum_local_consecutive_scores+=local_consecutive_score
                if local_consecutive_score > 0:
                    sum_local_consecutive_positive_scores+=local_consecutive_score
                elif local_consecutive_score < 0:
                    sum_local_consecutive_negative_scores+=local_consecutive_score
                else:
                    raise ValueError(f"Issue: local consecutive score should not be 0: {cur_consecutive_scores}")
            cur_consecutive_scores = [score]
            cur_consecutive_count = 0
            cur_consecutive_positive_count = 0
            cur_consecutive_negative_count = 0
        prev = score
    if len(cur_consecutive_scores) > 1:
        local_consecutive_score = find_consecutive_score(array(cur_consecutive_scores))*(len(cur_consecutive_scores)-1)
        sum_local_consecutive_scores+=local_consecutive_score
        if local_consecutive_score > 0:
            sum_local_consecutive_positive_scores+=local_consecutive_score
        elif local_consecutive_score < 0:
            sum_local_consecutive_negative_scores+=local_consecutive_score
        else:
            raise ValueError(f"Issue: local consecutive score should not be 0: {cur_consecutive_scores}")
    return total_consecutive_count, max_consecutive_count, sum_local_consecutive_scores, total_consecutive_positive_count, total_consecutive_negative_count, max_consecutive_positive_count, max_consecutive_negative_count, sum_local_consecutive_positive_scores, sum_local_consecutive_negative_scores

In [416]:
X5_total_consecutive_count_offset0 = []
X5_max_consecutive_count_offset0 = []
X5_sum_local_consecutive_scores_offset0 = []
X5_total_consecutive_positive_count_offset0 = []
X5_total_consecutive_negative_count_offset0 = []
X5_max_consecutive_positive_count_offest0 = []
X5_max_consecutive_negative_count_offset0 = []
X5_sum_local_consecutive_positive_scores_offset0 = []
X5_sum_local_consecutive_negative_scores_offset0 = []
for scores in X5_pooling_by_5_scores_offset0:
    (total_count, max_consecutive, sum_local_consecutive_scores, total_consecutive_positive_count, 
     total_consecutive_negative_count, max_consecutive_positive_count, max_consecutive_negative_count, 
     sum_local_consecutive_positive_scores, sum_local_consecutive_negative_scores) = analyze_consecutive(scores)
    X5_total_consecutive_count_offset0.append(total_count)
    X5_max_consecutive_count_offset0.append(max_consecutive)
    X5_sum_local_consecutive_scores_offset0.append(sum_local_consecutive_scores)
    X5_total_consecutive_positive_count_offset0.append(total_consecutive_positive_count)
    X5_total_consecutive_negative_count_offset0.append(total_consecutive_negative_count)
    X5_max_consecutive_positive_count_offest0.append(max_consecutive_positive_count)
    X5_max_consecutive_negative_count_offset0.append(max_consecutive_negative_count)
    X5_sum_local_consecutive_positive_scores_offset0.append(sum_local_consecutive_positive_scores)
    X5_sum_local_consecutive_negative_scores_offset0.append(sum_local_consecutive_negative_scores)
X5_total_consecutive_count_offset0 = array(X5_total_consecutive_count_offset0)
X5_max_consecutive_count_offset0 = array(X5_max_consecutive_count_offset0)
X5_sum_local_consecutive_scores_offset0 = array(X5_sum_local_consecutive_scores_offset0)
X5_total_consecutive_positive_count_offset0 = array(X5_total_consecutive_positive_count_offset0)
X5_total_consecutive_negative_count_offset0 = array(X5_total_consecutive_negative_count_offset0)
X5_max_consecutive_positive_count_offest0 = array(X5_max_consecutive_positive_count_offest0)
X5_max_consecutive_negative_count_offset0 = array(X5_max_consecutive_negative_count_offset0)
X5_sum_local_consecutive_positive_scores_offset0 = array(X5_sum_local_consecutive_positive_scores_offset0)
X5_sum_local_consecutive_negative_scores_offset0 = array(X5_sum_local_consecutive_negative_scores_offset0)

X5_total_consecutive_count_offset1 = []
X5_max_consecutive_count_offset1 = []
X5_sum_local_consecutive_scores_offset1 = []
for scores in X5_pooling_by_5_scores_offset1:
    (total_count, max_consecutive, sum_local_consecutive_scores,
     _,_,_,_,_,_) = analyze_consecutive(scores)
    X5_total_consecutive_count_offset1.append(total_count)
    X5_max_consecutive_count_offset1.append(max_consecutive)
    X5_sum_local_consecutive_scores_offset1.append(sum_local_consecutive_scores)
X5_total_consecutive_count_offset1 = array(X5_total_consecutive_count_offset1)
X5_max_consecutive_count_offset1 = array(X5_max_consecutive_count_offset1)
X5_sum_local_consecutive_scores_offset1 = array(X5_sum_local_consecutive_scores_offset1)

X5_total_consecutive_count_offset2 = []
X5_max_consecutive_count_offset2 = []
X5_sum_local_consecutive_scores_offset2 = []
for scores in X5_pooling_by_5_scores_offset2:
    (total_count, max_consecutive, sum_local_consecutive_scores,
     _,_,_,_,_,_) = analyze_consecutive(scores)
    X5_total_consecutive_count_offset2.append(total_count)
    X5_max_consecutive_count_offset2.append(max_consecutive)
    X5_sum_local_consecutive_scores_offset2.append(sum_local_consecutive_scores)
X5_total_consecutive_count_offset2 = array(X5_total_consecutive_count_offset2)
X5_max_consecutive_count_offset2 = array(X5_max_consecutive_count_offset2)
X5_sum_local_consecutive_scores_offset2 = array(X5_sum_local_consecutive_scores_offset2)

X5_total_consecutive_count_offset3 = []
X5_max_consecutive_count_offset3 = []
X5_sum_local_consecutive_scores_offset3 = []
for scores in X5_pooling_by_5_scores_offset3:
    (total_count, max_consecutive, sum_local_consecutive_scores,
     _,_,_,_,_,_) = analyze_consecutive(scores)
    X5_total_consecutive_count_offset3.append(total_count)
    X5_max_consecutive_count_offset3.append(max_consecutive)
    X5_sum_local_consecutive_scores_offset3.append(sum_local_consecutive_scores)
X5_total_consecutive_count_offset3 = array(X5_total_consecutive_count_offset3)
X5_max_consecutive_count_offset3 = array(X5_max_consecutive_count_offset3)
X5_sum_local_consecutive_scores_offset3 = array(X5_sum_local_consecutive_scores_offset3)

X5_total_consecutive_count_offset4 = []
X5_max_consecutive_count_offset4 = []
X5_sum_local_consecutive_scores_offset4 = []
for scores in X5_pooling_by_5_scores_offset4:
    (total_count, max_consecutive, sum_local_consecutive_scores,
     _,_,_,_,_,_) = analyze_consecutive(scores)
    X5_total_consecutive_count_offset4.append(total_count)
    X5_max_consecutive_count_offset4.append(max_consecutive)
    X5_sum_local_consecutive_scores_offset4.append(sum_local_consecutive_scores)
X5_total_consecutive_count_offset4 = array(X5_total_consecutive_count_offset4)
X5_max_consecutive_count_offset4 = array(X5_max_consecutive_count_offset4)
X5_sum_local_consecutive_scores_offset4 = array(X5_sum_local_consecutive_scores_offset4)

In [417]:
print(np.corrcoef(X5_total_consecutive_count_offset0, Y5)[0,1])
print(np.corrcoef(X5_total_consecutive_count_offset1, Y5)[0,1])
print(np.corrcoef(X5_total_consecutive_count_offset2, Y5)[0,1])
print(np.corrcoef(X5_total_consecutive_count_offset3, Y5)[0,1])
print(np.corrcoef(X5_total_consecutive_count_offset4, Y5)[0,1])

-0.08127688749723189
-0.07646272385558982
-0.06430264618695399
-0.05575659657763801
-0.06297143576096818


In [418]:
np.corrcoef(np.max(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0), Y5)[0,1]

-0.05121150083243527

In [419]:
np.corrcoef(np.mean(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0), Y5)[0,1]

-0.08415797928005254

In [420]:
np.corrcoef(np.min(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0), Y5)[0,1]

-0.10029144055544958

In [421]:
print(np.corrcoef(X5_max_consecutive_count_offset0, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_count_offset1, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_count_offset2, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_count_offset3, Y5)[0,1])
print(np.corrcoef(X5_max_consecutive_count_offset4, Y5)[0,1])

-0.059922220693694385
-0.0561972640740381
-0.04804998485633164
-0.039840396370631546
-0.045157037681160396


In [422]:
np.corrcoef(np.max(np.array([
    X5_max_consecutive_count_offset0, 
    X5_max_consecutive_count_offset1,
    X5_max_consecutive_count_offset2,
    X5_max_consecutive_count_offset3,
    X5_max_consecutive_count_offset4,
    ]), axis=0), Y5)[0,1]

-0.04235348290040901

In [423]:
np.corrcoef(np.mean(np.array([
    X5_max_consecutive_count_offset0, 
    X5_max_consecutive_count_offset1,
    X5_max_consecutive_count_offset2,
    X5_max_consecutive_count_offset3,
    X5_max_consecutive_count_offset4,
    ]), axis=0), Y5)[0,1]

-0.05922283956103924

In [424]:
np.corrcoef(np.min(np.array([
    X5_max_consecutive_count_offset0, 
    X5_max_consecutive_count_offset1,
    X5_max_consecutive_count_offset2,
    X5_max_consecutive_count_offset3,
    X5_max_consecutive_count_offset4,
    ]), axis=0), Y5)[0,1]

-0.06236494807027054

In [425]:
print(np.corrcoef(X5_sum_local_consecutive_scores_offset0, Y5)[0,1])
print(np.corrcoef(X5_sum_local_consecutive_scores_offset1, Y5)[0,1])
print(np.corrcoef(X5_sum_local_consecutive_scores_offset2, Y5)[0,1])
print(np.corrcoef(X5_sum_local_consecutive_scores_offset3, Y5)[0,1])
print(np.corrcoef(X5_sum_local_consecutive_scores_offset4, Y5)[0,1])

-0.059883988656981274
-0.05722317059937096
-0.05383737008602341
-0.04813002334662755
-0.05196925798697151


In [426]:
np.corrcoef(np.max(np.array([
    X5_sum_local_consecutive_scores_offset0, 
    X5_sum_local_consecutive_scores_offset1,
    X5_sum_local_consecutive_scores_offset2,
    X5_sum_local_consecutive_scores_offset3,
    X5_sum_local_consecutive_scores_offset4,
    ]), axis=0), Y5)[0,1]

-0.048283182052152825

In [427]:
np.corrcoef(np.mean(np.array([
    X5_sum_local_consecutive_scores_offset0, 
    X5_sum_local_consecutive_scores_offset1,
    X5_sum_local_consecutive_scores_offset2,
    X5_sum_local_consecutive_scores_offset3,
    X5_sum_local_consecutive_scores_offset4,
    ]), axis=0), Y5)[0,1]

-0.05769649473272988

In [428]:
np.corrcoef(np.min(np.array([
    X5_sum_local_consecutive_scores_offset0, 
    X5_sum_local_consecutive_scores_offset1,
    X5_sum_local_consecutive_scores_offset2,
    X5_sum_local_consecutive_scores_offset3,
    X5_sum_local_consecutive_scores_offset4,
    ]), axis=0), Y5)[0,1]

-0.0641374580053486

In [429]:
print(np.corrcoef(X5_total_consecutive_positive_count_offset0, Y5)[0,1])

-0.06569358267464308


In [430]:
print(np.corrcoef(X5_total_consecutive_negative_count_offset0, Y5)[0,1])

0.0020466502970163526


In [431]:
print(np.corrcoef(X5_max_consecutive_positive_count_offest0, Y5)[0,1])

-0.0630929455076141


In [432]:
print(np.corrcoef(X5_max_consecutive_negative_count_offset0, Y5)[0,1])

0.0047462147397506725


In [433]:
print(np.corrcoef(X5_sum_local_consecutive_positive_scores_offset0, Y5)[0,1])

-0.06107847291145367


In [434]:
print(np.corrcoef(X5_sum_local_consecutive_negative_scores_offset0, Y5)[0,1])

-0.013728403417304623


## Regression

In [435]:
X5_global_alternating_scores_max = np.max(np.array([
    X5_global_alternating_scores_offset0, 
    X5_global_alternating_scores_offset1,
    X5_global_alternating_scores_offset2,
    X5_global_alternating_scores_offset3,
    X5_global_alternating_scores_offset4,
    ]), axis=0)

X5_global_alternating_scores_mean = np.mean(np.array([
    X5_global_alternating_scores_offset0, 
    X5_global_alternating_scores_offset1,
    X5_global_alternating_scores_offset2,
    X5_global_alternating_scores_offset3,
    X5_global_alternating_scores_offset4,
    ]), axis=0)

X5_global_alternating_scores_min = np.min(np.array([
    X5_global_alternating_scores_offset0, 
    X5_global_alternating_scores_offset1,
    X5_global_alternating_scores_offset2,
    X5_global_alternating_scores_offset3,
    X5_global_alternating_scores_offset4,
    ]), axis=0)

In [436]:
X5_count_alternating_max = np.max(np.array([
    X5_count_alternating_offset0, 
    X5_count_alternating_offset1,
    X5_count_alternating_offset2,
    X5_count_alternating_offset3,
    X5_count_alternating_offset4,
    ]), axis=0)

X5_count_alternating_mean = np.mean(np.array([
    X5_count_alternating_offset0, 
    X5_count_alternating_offset1,
    X5_count_alternating_offset2,
    X5_count_alternating_offset3,
    X5_count_alternating_offset4,
    ]), axis=0)

X5_count_alternating_min = np.min(np.array([
    X5_count_alternating_offset0, 
    X5_count_alternating_offset1,
    X5_count_alternating_offset2,
    X5_count_alternating_offset3,
    X5_count_alternating_offset4,
    ]), axis=0)

In [437]:
X5_max_consecutive_alternating_max = np.max(np.array([
    X5_max_consecutive_alternating_offset0, 
    X5_max_consecutive_alternating_offset1,
    X5_max_consecutive_alternating_offset2,
    X5_max_consecutive_alternating_offset3,
    X5_max_consecutive_alternating_offset4,
    ]), axis=0)

X5_max_consecutive_alternating_mean = np.mean(np.array([
    X5_max_consecutive_alternating_offset0, 
    X5_max_consecutive_alternating_offset1,
    X5_max_consecutive_alternating_offset2,
    X5_max_consecutive_alternating_offset3,
    X5_max_consecutive_alternating_offset4,
    ]), axis=0)

X5_max_consecutive_alternating_offset0 = np.min(np.array([
    X5_max_consecutive_alternating_offset0, 
    X5_max_consecutive_alternating_offset1,
    X5_max_consecutive_alternating_offset2,
    X5_max_consecutive_alternating_offset3,
    X5_max_consecutive_alternating_offset4,
    ]), axis=0)

In [438]:
X5_sum_local_alternating_scores_max = np.max(np.array([
    X5_sum_local_alternating_scores_offset0, 
    X5_sum_local_alternating_scores_offset1,
    X5_sum_local_alternating_scores_offset2,
    X5_sum_local_alternating_scores_offset3,
    X5_sum_local_alternating_scores_offset4,
    ]), axis=0)

X5_sum_local_alternating_scores_mean = np.mean(np.array([
    X5_sum_local_alternating_scores_offset0, 
    X5_sum_local_alternating_scores_offset1,
    X5_sum_local_alternating_scores_offset2,
    X5_sum_local_alternating_scores_offset3,
    X5_sum_local_alternating_scores_offset4,
    ]), axis=0)

X5_sum_local_alternating_scores_min = np.min(np.array([
    X5_sum_local_alternating_scores_offset0, 
    X5_sum_local_alternating_scores_offset1,
    X5_sum_local_alternating_scores_offset2,
    X5_sum_local_alternating_scores_offset3,
    X5_sum_local_alternating_scores_offset4,
    ]), axis=0)

In [439]:
X5_total_consecutive_count_max = np.max(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0)

In [440]:
X5_total_consecutive_count_max = np.max(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0)

X5_total_consecutive_count_mean = np.mean(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0)

X5_total_consecutive_count_min = np.min(np.array([
    X5_total_consecutive_count_offset0, 
    X5_total_consecutive_count_offset1,
    X5_total_consecutive_count_offset2,
    X5_total_consecutive_count_offset3,
    X5_total_consecutive_count_offset4,
    ]), axis=0)

In [448]:
X5_max_consecutive_count_max = np.max(np.array([
    X5_max_consecutive_count_offset0, 
    X5_max_consecutive_count_offset1,
    X5_max_consecutive_count_offset2,
    X5_max_consecutive_count_offset3,
    X5_max_consecutive_count_offset4,
    ]), axis=0)

X5_max_consecutive_count_mean = np.mean(np.array([
    X5_max_consecutive_count_offset0, 
    X5_max_consecutive_count_offset1,
    X5_max_consecutive_count_offset2,
    X5_max_consecutive_count_offset3,
    X5_max_consecutive_count_offset4,
    ]), axis=0)

X5_max_consecutive_count_min = np.min(np.array([
    X5_max_consecutive_count_offset0, 
    X5_max_consecutive_count_offset1,
    X5_max_consecutive_count_offset2,
    X5_max_consecutive_count_offset3,
    X5_max_consecutive_count_offset4,
    ]), axis=0)

In [456]:
X5_sum_local_consecutive_scores_max = np.max(np.array([
    X5_sum_local_consecutive_scores_offset0, 
    X5_sum_local_consecutive_scores_offset1,
    X5_sum_local_consecutive_scores_offset2,
    X5_sum_local_consecutive_scores_offset3,
    X5_sum_local_consecutive_scores_offset4,
    ]), axis=0)

X5_sum_local_consecutive_scores_mean = np.mean(np.array([
    X5_sum_local_consecutive_scores_offset0, 
    X5_sum_local_consecutive_scores_offset1,
    X5_sum_local_consecutive_scores_offset2,
    X5_sum_local_consecutive_scores_offset3,
    X5_sum_local_consecutive_scores_offset4,
    ]), axis=0)

X5_sum_local_consecutive_scores_min = np.min(np.array([
    X5_sum_local_consecutive_scores_offset0, 
    X5_sum_local_consecutive_scores_offset1,
    X5_sum_local_consecutive_scores_offset2,
    X5_sum_local_consecutive_scores_offset3,
    X5_sum_local_consecutive_scores_offset4,
    ]), axis=0)

In [462]:
X5_new = pd.DataFrame([X5_global_alternating_scores_max, X5_global_alternating_scores_mean, X5_global_alternating_scores_min, 
                       X5_count_alternating_max, X5_count_alternating_mean, X5_count_alternating_min, 
                       X5_max_consecutive_alternating_max, X5_max_consecutive_alternating_mean, X5_max_consecutive_alternating_offset0,
                       X5_sum_local_alternating_scores_max, X5_sum_local_alternating_scores_mean, X5_sum_local_alternating_scores_min,
                       X5_global_consecutive_scores,
                       X5_total_consecutive_count_max, X5_total_consecutive_count_mean, X5_total_consecutive_count_min,
                       X5_max_consecutive_count_max, X5_max_consecutive_count_mean, X5_max_consecutive_count_min,
                       X5_sum_local_consecutive_scores_max, X5_sum_local_consecutive_scores_mean, X5_sum_local_consecutive_scores_min,
                       X5_total_consecutive_positive_count_offset0,
                       X5_total_consecutive_negative_count_offset0,
                       X5_max_consecutive_positive_count_offest0,
                       X5_max_consecutive_negative_count_offset0,
                       X5_sum_local_consecutive_positive_scores_offset0,
                       X5_sum_local_consecutive_negative_scores_offset0]).transpose()

In [463]:
X5_new

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,6.0,3.6,2.0,5.0,4.0,3.0,4.0,2.4,2.0,24.0,...,2.0,34.0,26.0,16.0,4.0,0.0,2.0,0.0,16.0,0.0
1,10.0,4.4,2.0,6.0,4.6,2.0,5.0,2.6,2.0,45.0,...,2.0,44.0,22.0,14.0,3.0,0.0,2.0,0.0,14.0,0.0
2,10.0,5.2,2.0,6.0,4.8,2.0,5.0,3.0,1.0,44.0,...,2.0,38.0,16.4,8.0,3.0,1.0,3.0,1.0,12.0,-4.0
3,10.0,6.8,2.0,8.0,5.8,2.0,6.0,3.6,1.0,79.0,...,1.0,28.0,9.0,2.0,2.0,1.0,2.0,1.0,6.0,-2.0
4,6.0,3.6,2.0,7.0,5.8,3.0,5.0,3.6,2.0,54.0,...,1.0,14.0,3.0,-2.0,4.0,2.0,4.0,1.0,20.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82363,14.0,6.8,2.0,4.0,2.4,2.0,2.0,1.8,1.0,27.0,...,4.0,100.0,82.6,62.0,7.0,0.0,4.0,0.0,82.0,0.0
82364,10.0,5.2,2.0,4.0,2.4,2.0,2.0,1.8,1.0,24.0,...,2.0,142.0,110.2,46.0,5.0,0.0,2.0,0.0,46.0,0.0
82365,12.0,8.0,0.0,5.0,3.4,3.0,2.0,1.8,1.0,30.0,...,2.0,131.0,98.8,33.0,6.0,0.0,6.0,0.0,114.0,0.0
82366,16.0,8.8,4.0,5.0,3.4,3.0,2.0,1.6,1.0,34.0,...,2.0,128.0,94.6,24.0,6.0,0.0,6.0,0.0,126.0,0.0


In [464]:
from sklearn import linear_model

In [465]:
regr = linear_model.LinearRegression()
regr.fit(X5_new, Y5)

LinearRegression()

In [466]:
np.corrcoef(regr.predict(X5_new), Y5)[0,1]

0.40052723209327656

In [499]:
import xgboost as xgb
from sklearn.base import clone
from sklearn.model_selection import KFold

In [500]:
def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score

In [501]:
cv = KFold(n_splits=10, shuffle=True, random_state=50)
reg = xgb.XGBRegressor(tree_method="hist")

In [508]:
results = {}

for train, test in cv.split(X5_new, Y5):
    print(f"New Train-test split")
    X_train = array(X5_new)[train,:]
    X_test = array(X5_new)[test,:]
    y_train = Y5[train]
    y_test = Y5[test]
    est, train_score, test_score = fit_and_score(
        clone(reg), X_train, X_test, y_train, y_test
    )
    results[est] = (train_score, test_score)
    print(f"Correlation: {np.corrcoef(est.predict(X_test), y_test)[0,1]}")

New Train-test split
[0]	validation_0-rmse:0.47222
[1]	validation_0-rmse:0.46076
[2]	validation_0-rmse:0.45466
[3]	validation_0-rmse:0.45178
[4]	validation_0-rmse:0.44999
[5]	validation_0-rmse:0.44889
[6]	validation_0-rmse:0.44855
[7]	validation_0-rmse:0.44820
[8]	validation_0-rmse:0.44802
[9]	validation_0-rmse:0.44813
[10]	validation_0-rmse:0.44806
[11]	validation_0-rmse:0.44804
[12]	validation_0-rmse:0.44795
[13]	validation_0-rmse:0.44809
[14]	validation_0-rmse:0.44816
[15]	validation_0-rmse:0.44808
[16]	validation_0-rmse:0.44812
[17]	validation_0-rmse:0.44792
[18]	validation_0-rmse:0.44779
[19]	validation_0-rmse:0.44787
[20]	validation_0-rmse:0.44791
[21]	validation_0-rmse:0.44788
[22]	validation_0-rmse:0.44785
[23]	validation_0-rmse:0.44800
[24]	validation_0-rmse:0.44785
[25]	validation_0-rmse:0.44786
[26]	validation_0-rmse:0.44798
[27]	validation_0-rmse:0.44796
[28]	validation_0-rmse:0.44795
[29]	validation_0-rmse:0.44802
[30]	validation_0-rmse:0.44809
[31]	validation_0-rmse:0.448

In [505]:
np.corrcoef(est.predict(array(X5_new)), Y5)[0,1]

0.5486072440588302

In [495]:
np.corrcoef(reg.predict(X5_new), Y5)[0,1]

NotFittedError: need to call fit or load_model beforehand

In [471]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'regression'}
param['eval_metric'] = 'mse'
evallist = [(X5_new_xgb_train, 'train')]

num_round = 10
bst = xgb.train(param, X5_new_xgb_train, num_round, evallist)

/Users/Brody1/anaconda3/envs/gnns/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBoostError: [20:34:26] /Users/runner/work/xgboost/xgboost/src/objective/objective.cc:27: Unknown objective function: `regression`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: rank:ndcg
Objective candidate: rank:pairwise
Objective candidate: rank:map
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: reg:quantileerror
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: reg:pseudohubererror
Objective candidate: count:poisson
Objective candidate: survival:cox
Objective candidate: reg:gamma
Objective candidate: reg:tweedie
Objective candidate: reg:absoluteerror

Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000157376ab5 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dyl